# regularization 


## stopping early.

- stop the training when the validation loss does not improve anymore.
  - hyper parameter
      - the number of validation loss does not improve
- need cache for validation loss.
- validation loss can be calculated as parallel to the training loss.
- model or class does not have to be changed.

## Dataset augmentation

-  dataset augumentation.
  - eg)
    - image
      - upside down
      - masking
      - colorless
      - etc... 
- inject noise into the network
-  label smoothing.




## add norm penalty to the loss function

loss_hat = loss + penalty(w)   

- lasso
  - / w /
  - work as variable selection
    - but if multiple variable has correlation, lasso try to reduce all of them
- ridge
  - transpose(w) * w / 2
  - make parameter close to zero.
  -  when you already know the parameter will get close to a certain parameter
    - // w - b //^2
    - or
    - // w(1) - w(2) //^2
- elastic net
  - the combination of ridge and lasso
  -  work as variable selection
    - if multiple variable has the correlation, pick some of them to reduce.
## multi task learning

- when multiple tasks has interactions between each others.
  - if one task lacks of something
  - but
  - the other has the information
  - and also
  - there is a interaction between them
  - the model can improve based on the task that has the information 

- traditionally, it is common to use each model for each task.
- use
  - common part
  - relation
  - correlation
  - between multiple tasks
    - so that the model can deal with multiple tasks at the same time.

**benefit**

- the model can share the multiple tasks from the multiple tasks.

- reuse the data and the resource.
- the interaction between multi tasks.
  - the improvement for both task`s performance
- reduce the number of labels.

**choose and construct tasks**

- difficulity of tasks.
  - the difficulity of task should be similar
  - if some of them are easy and some of them are difficult, then the model will focus on easier one more. 
- order of tasks
- collecting data and preprocess.

## transfer training

- use model that is already trained with some train data
  - to learn another train data with additional information.
- the size of train data is small
- and
- the task is similar.

- when the size of train data is large
  - you can tune more new layer by using transfer learning.
 
## Esemble learninig

- train weaker models
- and
- average/aggregate the reuslt
  - classification - mode
  - regression - average.
 
- if models are independent
  - they tend not to make the same error
  -  the model can learn some variety of error.
  -  

- tree
  - random forest tree

### baggin (one methods for esemble learning) 

- bootstrap
- and
- aggregating
1. bootstrappin the k data set
2. train k different models for k data set.
3. aggregating k results from the k different models.

- for neural network
  - just changing the hyper parameter can be considered to be different models.
  - but
  - it takes a lot of time if you want to implement the ensemble learning for neural network
    -  you can get each snapshot of local minimum and average them
   
## drop out

- for neural network
- like baggin within signle neural network
- by drop some of the units in some layers.

**how**

- arrange the mask vector
  - each element is from bernoulli distribution
    -  1 with p=p or 0 with p=1-p
    -  usually
      - p = 0.8 for input layer
      - p = 0.5 for hidden layer.
   
**training**

- after you get the z(i)
- you create the mask(i) whose shape is the same as z(i)
- multiply mask(i) to z(i)

**testing**

- after many times of learning
- the contribution of w(i)h(i) is just p * w(i) h(i)
-  so for testing
-  you just have to multiply the affine with p
-  or
-  multiple the activate(affine()) with p

**inverted dropout**

- maintain the expectation of output as same as without dropout.

# Multtask learning

In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms


transform = transforms.Compose([transforms.ToTensor()])

train_set = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_set = datasets.MNIST('./data', train=False, transform=transform)

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)


class Multitask_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.digit_fc = nn.Linear(32, 10) 
        self.parity_fc = nn.Linear(32, 2)
        
    def forward(self, x):
        x = x.view(-1, 784) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        digit_logit = self.digit_fc(x)
        parity_logit = self.parity_fc(x)
        return digit_logit, parity_logit

model = Multitask_Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


num_epochs = 10
loss_history = []

for epoch in range(num_epochs):

    running_loss = 0
    
    for batch in train_loader:
        img, label = batch
        parity_label = label % 2
        
        optimizer.zero_grad()
        
        digit_logit, parity_logit = model(img)
        
        digit_loss = criterion(digit_logit, label) 
        parity_loss = criterion(parity_logit, parity_label)
        
        loss = digit_loss + parity_loss
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()

    avg_loss = running_loss / len(train_loader)
    loss_history.append(avg_loss)
    print(f'epoch: {epoch}, loss: {avg_loss}')


plt.plot(loss_history)
plt.title('Training Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

ModuleNotFoundError: No module named 'torch'

In [1]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


# dropout

In [8]:
p = 0.5
relu = lambda x: x * (x>0) 

def forward(x):
    a1 = np.dot(w1,x) + b1
    z1 = relu(a1)
    # get the usual output for layer 1 withtout dropout
    m1 = np.random.rand(*z1.shape)<p
    # get the mask whose shape is the same as z1
    z1 *= m1
    # multiply the 


    a2 = np.dot(w2, z1)+b2
    z2 = relu(a2)
    m2 = np.random.rand(*z2.shape) <p
    z2 *=m2

    out = np.dot(w3,z2)+b3
    return out


def test(X):
    
    a1 = np.dot(w1,x) + b1
    z1 = relu(a1) * p
    a2 = np.dot(w2, z1)+b2
    z2 = relu(a2) * p
    out = np.dot(w3,z2)+b3
    return out
    

# inverted drop out

In [13]:
p = 0.5
relu = lambda x: x*(x>0)

def forward(x):
    a1 = np.dot(w1,x) + b1
    z1 = relu(a1)
    m1 = (np.random.rand(*z1.shape) <p )/ p
    z1 *= m1

    a2 = np.dot(w2,z1) + b2
    z2 = relu(a2)
    m2 = (np.random.rand(*z2.shape) <p ) / p
    z2 *= m2

    out = np.dot(w3,z2) + b3

def test(X):
    a1 = np.dot(w1,x) + b1
    z1 = relu(a1)
    a2 = np.dot(w2,z1) + b2
    z2 = relu(a2)
    out = np.dot(w3, z2) + b3
    return out